In [2]:
import pandas as pd
import pandas as pd
import matplotlib.pyplot as plt
from sentence_transformers import SentenceTransformer, util
from PIL import Image
import glob
import torch
import numpy as np
import matplotlib.pyplot as plt
import os
from data_load import load_data

c:\Users\kiemd\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [11]:
df,images=load_data('./data/data_csv')

In [13]:
df.shape

(6324, 8)

In [14]:
df.duplicated().sum()

0

In [15]:
df.isnull().sum()

Nom              0
prix             0
link             0
marque           0
Description      0
modele          22
couleur        498
cathegorie       0
dtype: int64

In [16]:
df.dropna(inplace=True)

In [17]:
df.isnull().sum()

Nom            0
prix           0
link           0
marque         0
Description    0
modele         0
couleur        0
cathegorie     0
dtype: int64

In [18]:
df.shape

(5808, 8)

In [19]:
df['identification'] = range(len(df))

In [20]:
df.head()

,Nom,prix,link,marque,Description,modele,couleur,cathegorie,identification
0,Generic Tondeuse HTC pour Barbe et tête profes...,2500,https://sn.jumia.is/unsafe/fit-in/300x300/filt...,Generic,"['Qualité professionnelle', 'Rechargeable et à...",HTC,Blanc,beaute-hygiene-sante,0
1,Nova Gel Douche Classic - 300ML,480,https://sn.jumia.is/unsafe/fit-in/300x300/filt...,Nova,"['Type Produit : Soin Corps', 'Marque : Nova',...",1,Jumia Mall,beaute-hygiene-sante,1
3,Day By Day Men Déodorant Bille Roll On Men Cla...,490,https://sn.jumia.is/unsafe/fit-in/300x300/filt...,Day By Day Men,"['Type De Produit: Hygiène Corps', 'Marque:\xa...",1,bleu,beaute-hygiene-sante,2
4,Lanbena Liquide blanchissant dentaire biologiq...,1350,https://sn.jumia.is/unsafe/fit-in/300x300/filt...,Lanbena,"['Effet blanchissant.', 'Efficace pour enlever...",Lb3879,blanche,beaute-hygiene-sante,3
5,Shower to Shower Déodorant Bille Antitranspira...,790,https://sn.jumia.is/unsafe/fit-in/300x300/filt...,Shower to Shower,"['Type Produit : Déodorant', 'Marque : Shower ...",.1,Rose,beaute-hygiene-sante,4


In [11]:
df.shape

(5808, 9)

In [21]:
import os
import requests
import pandas as pd
import re


# Spécifier le dossier de destination pour les images (chemin absolu)
output_folder = os.path.abspath("data1_image")

# Créer le dossier s'il n'existe pas
os.makedirs(output_folder, exist_ok=True)

# Liste pour stocker les URLs des images qui n'ont pas pu être téléchargées
failed_urls = []

# Fonction pour nettoyer le nom du fichier
def clean_filename(filename):
    # Supprimer les caractères spéciaux à l'aide d'une expression régulière
    cleaned_filename = re.sub(r"[^a-zA-Z0-9_\-\.]", "_", filename)
    return cleaned_filename

# Fonction pour télécharger une image à partir d'un lien et l'enregistrer dans le dossier
def download_image(row, output_folder):
    try:
        image_name = f"{row['identification']}_{row['Nom']}"  # Utiliser l'identification dans le nom du fichier
        image_url = row['link']
        response = requests.get(image_url)
        if response.status_code == 200:
            # Nettoyer le nom du fichier
            cleaned_image_name = clean_filename(image_name)

            # Créer le sous-dossier correspondant à la catégorie si nécessaire
            category_folder = os.path.join(output_folder, row['cathegorie'])
            os.makedirs(category_folder, exist_ok=True)

            filename = os.path.join(category_folder, f"{cleaned_image_name}.jpg")
            df.at[row.name, 'file_name'] = filename  # Utiliser l'index pour mettre à jour la valeur dans le DataFrame
            with open(filename, 'wb') as file:
                file.write(response.content)
            print(f"Image téléchargée : {filename}")
        else:
            print(f"Échec du téléchargement de l'image à partir de {image_url}. Statut {response.status_code}")
            failed_urls.append(image_url)
    except Exception as e:
        print(f"Erreur lors du téléchargement de l'image à partir de {image_url}: {e}")
        failed_urls.append(image_url)

# Assurez-vous qu'il n'y a pas de valeurs manquantes ou de doublons dans la colonne 'identification'
df = df.dropna(subset=['identification']).drop_duplicates(subset=['identification'])

# Triez le DataFrame en fonction de la colonne 'identification'
df.sort_values(by='identification', inplace=True)

# Réinitialisez l'index du DataFrame
df.reset_index(drop=True, inplace=True)

# Ajouter une colonne 'file_name' vide au DataFrame
df['file_name'] = ''

# Appliquez la fonction pour chaque ligne du DataFrame
total_images_downloaded = 0
for index, row in df.iterrows():
    try:
        download_image(row, output_folder)
        total_images_downloaded += 1
    except Exception as e:
        print(f"Erreur lors du traitement de la ligne {index}: {e}")

# Vérifiez si la colonne 'file_name' a été ajoutée
print(df[['identification', 'file_name']].head())

# Imprimez le nombre total d'images téléchargées
print(f"Nombre total d'images téléchargées : {total_images_downloaded}")

# Imprimez les URLs des images qui n'ont pas pu être téléchargées
print("URLs des images qui n'ont pas pu être téléchargées :")
for url in failed_urls:
    print(url)


Image téléchargée : c:\Users\kiemd\OneDrive\Desktop\Porjetfin\data1_image\beaute-hygiene-sante\0_Generic_Tondeuse_HTC_pour_Barbe_et_t_te_professionnelle__-_AT_206.jpg
Image téléchargée : c:\Users\kiemd\OneDrive\Desktop\Porjetfin\data1_image\beaute-hygiene-sante\1_Nova_Gel_Douche_Classic_-_300ML.jpg
Image téléchargée : c:\Users\kiemd\OneDrive\Desktop\Porjetfin\data1_image\beaute-hygiene-sante\2_Day_By_Day_Men_D_odorant_Bille_Roll_On_Men_Classic_Vert_50_Ml.jpg
Image téléchargée : c:\Users\kiemd\OneDrive\Desktop\Porjetfin\data1_image\beaute-hygiene-sante\3_Lanbena_Liquide_blanchissant_dentaire_biologique_-_10_ml.jpg
Image téléchargée : c:\Users\kiemd\OneDrive\Desktop\Porjetfin\data1_image\beaute-hygiene-sante\4_Shower_to_Shower_D_odorant_Bille_Antitranspirant_48h__Musk_Alive.jpg
Image téléchargée : c:\Users\kiemd\OneDrive\Desktop\Porjetfin\data1_image\beaute-hygiene-sante\5_Generic_Tondeuse___Cheveux_Electrique_Rechargeable__-_Noir.jpg
Image téléchargée : c:\Users\kiemd\OneDrive\Desktop\P

In [22]:
total_images_downloaded

5808

In [23]:
df.to_csv('data_vrai.csv',index=False)

In [27]:
df['Nom'][5805]

'Samsung Galaxy A03 - 4G - 6.5" - 4Go - 64Go - 48Mpx - 5000 MAh – Bleu'

In [28]:
df['file_name'][5805]

'c:\\Users\\kiemd\\OneDrive\\Desktop\\Porjetfin\\data1_image\\telephone-tablette\\5805_Samsung_Galaxy_A03_-_4G_-_6.5__-_4Go_-_64Go_-_48Mpx_-_5000_MAh___Bleu.jpg'

In [24]:
failed_urls

['https://sn.jumia.is/unsafe/fit-in/300x300/filters:fill(white)/product/02/558021/1.jpg?0406',
 'https://sn.jumia.is/unsafe/fit-in/300x300/filters:fill(white)/product/93/174121/1.jpg?3365',
 'https://sn.jumia.is/unsafe/fit-in/300x300/filters:fill(white)/product/50/764121/1.jpg?8641',
 'https://sn.jumia.is/unsafe/fit-in/300x300/filters:fill(white)/product/80/171021/1.jpg?4075',
 'https://sn.jumia.is/unsafe/fit-in/300x300/filters:fill(white)/product/17/147811/1.jpg?4631',
 'https://sn.jumia.is/unsafe/fit-in/300x300/filters:fill(white)/product/12/373121/1.jpg?4750',
 'https://sn.jumia.is/unsafe/fit-in/300x300/filters:fill(white)/product/41/673121/1.jpg?9246',
 'https://sn.jumia.is/unsafe/fit-in/300x300/filters:fill(white)/product/21/673121/1.jpg?9246',
 'https://sn.jumia.is/unsafe/fit-in/300x300/filters:fill(white)/product/11/673121/1.jpg?9246',
 'https://sn.jumia.is/unsafe/fit-in/300x300/filters:fill(white)/product/66/568121/1.jpg?6235',
 'https://sn.jumia.is/unsafe/fit-in/300x300/filter

In [15]:
df.head()

,Nom,prix,link,marque,Description,modele,couleur,cathegorie,identification,file_name
0,Generic Tondeuse HTC pour Barbe et tête profes...,2500,https://sn.jumia.is/unsafe/fit-in/300x300/filt...,Generic,"['Qualité professionnelle', 'Rechargeable et à...",HTC,Blanc,beaute-hygiene-sante,0,c:\Users\kiemd\OneDrive\Desktop\Porjetfin\data...
1,Nova Gel Douche Classic - 300ML,480,https://sn.jumia.is/unsafe/fit-in/300x300/filt...,Nova,"['Type Produit : Soin Corps', 'Marque : Nova',...",1,Jumia Mall,beaute-hygiene-sante,1,c:\Users\kiemd\OneDrive\Desktop\Porjetfin\data...
2,Day By Day Men Déodorant Bille Roll On Men Cla...,490,https://sn.jumia.is/unsafe/fit-in/300x300/filt...,Day By Day Men,"['Type De Produit: Hygiène Corps', 'Marque:\xa...",1,bleu,beaute-hygiene-sante,3,c:\Users\kiemd\OneDrive\Desktop\Porjetfin\data...
3,Lanbena Liquide blanchissant dentaire biologiq...,1350,https://sn.jumia.is/unsafe/fit-in/300x300/filt...,Lanbena,"['Effet blanchissant.', 'Efficace pour enlever...",Lb3879,blanche,beaute-hygiene-sante,4,c:\Users\kiemd\OneDrive\Desktop\Porjetfin\data...
4,Shower to Shower Déodorant Bille Antitranspira...,790,https://sn.jumia.is/unsafe/fit-in/300x300/filt...,Shower to Shower,"['Type Produit : Déodorant', 'Marque : Shower ...",.1,Rose,beaute-hygiene-sante,5,c:\Users\kiemd\OneDrive\Desktop\Porjetfin\data...


In [40]:
import os
import requests
import pandas as pd
import re

# Charger votre DataFrame avec les liens d'images
# df = ...

# Spécifier le dossier de destination pour les images (chemin absolu)
output_folder = os.path.abspath("data_images")

# Créer le dossier s'il n'existe pas
os.makedirs(output_folder, exist_ok=True)

# Fonction pour nettoyer le nom du fichier
def clean_filename(filename):
    # Supprimer les caractères spéciaux à l'aide d'une expression régulière
    cleaned_filename = re.sub(r"[^a-zA-Z0-9_\-\.]", "_", filename)
    return cleaned_filename

# Fonction pour télécharger une image à partir d'un lien et l'enregistrer dans le dossier
def download_image(row, output_folder):
    try:
        image_name = f"{row['identification']}_{row['Nom']}"  # Utiliser l'identification dans le nom du fichier
        image_url = row['link']
        response = requests.get(image_url)
        if response.status_code == 200:
            # Nettoyer le nom du fichier
            cleaned_image_name = clean_filename(image_name)

            # Créer le sous-dossier correspondant à la catégorie si nécessaire
            category_folder = os.path.join(output_folder, row['cathegorie'])
            os.makedirs(category_folder, exist_ok=True)

            filename = os.path.join(category_folder, f"{cleaned_image_name}.jpg")
            df.at[row.name, 'file_name'] = filename  # Utiliser l'index pour mettre à jour la valeur dans le DataFrame
            with open(filename, 'wb') as file:
                file.write(response.content)
            print(f"Image téléchargée : {filename}")
        else:
            print(f"Échec du téléchargement de l'image à partir de {image_url}. Statut {response.status_code}")
    except Exception as e:
        print(f"Erreur lors du téléchargement de l'image à partir de {image_url}: {e}")

# Assurez-vous qu'il n'y a pas de valeurs manquantes ou de doublons dans la colonne 'identification'
df = df.dropna(subset=['identification']).drop_duplicates(subset=['identification'])

# Triez le DataFrame en fonction de la colonne 'identification'
df.sort_values(by='identification', inplace=True)

# Réinitialisez l'index du DataFrame
df.reset_index(drop=True, inplace=True)

# Ajouter une colonne 'file_name' vide au DataFrame
df['file_name'] = ''

# Appliquez la fonction pour chaque ligne du DataFrame
for index, row in df.iterrows():
    download_image(row, output_folder)

# Vérifiez si la colonne 'file_name' a été ajoutée
print(df[['identification', 'file_name']].head())


Image téléchargée : c:\Users\kiemd\OneDrive\Desktop\Porjetfin\data_images\beaute-hygiene-sante\0_Generic_Tondeuse_HTC_pour_Barbe_et_t_te_professionnelle__-_AT_206.jpg
Image téléchargée : c:\Users\kiemd\OneDrive\Desktop\Porjetfin\data_images\beaute-hygiene-sante\1_Nova_Gel_Douche_Classic_-_300ML.jpg
Image téléchargée : c:\Users\kiemd\OneDrive\Desktop\Porjetfin\data_images\beaute-hygiene-sante\3_Day_By_Day_Men_D_odorant_Bille_Roll_On_Men_Classic_Vert_50_Ml.jpg
Image téléchargée : c:\Users\kiemd\OneDrive\Desktop\Porjetfin\data_images\beaute-hygiene-sante\4_Lanbena_Liquide_blanchissant_dentaire_biologique_-_10_ml.jpg
Image téléchargée : c:\Users\kiemd\OneDrive\Desktop\Porjetfin\data_images\beaute-hygiene-sante\5_Shower_to_Shower_D_odorant_Bille_Antitranspirant_48h__Musk_Alive.jpg
Image téléchargée : c:\Users\kiemd\OneDrive\Desktop\Porjetfin\data_images\beaute-hygiene-sante\6_Generic_Tondeuse___Cheveux_Electrique_Rechargeable__-_Noir.jpg
Image téléchargée : c:\Users\kiemd\OneDrive\Desktop\P

In [45]:
df.to_csv('./data/data_conta.csv',index=False)

In [8]:
import os
import requests
import pandas as pd
import re  # Importer le module de expressions régulières

# Charger votre DataFrame avec les liens d'images
# df = ...

# Spécifier le dossier de destination pour les images (chemin absolu)
output_folder = os.path.abspath("images2/telephone_tablette")

# Créer le dossier s'il n'existe pas
os.makedirs(output_folder, exist_ok=True)

# Fonction pour nettoyer le nom du fichier
def clean_filename(filename):
    # Supprimer les caractères spéciaux à l'aide d'une expression régulière
    cleaned_filename = re.sub(r"[^a-zA-Z0-9_\-\.]", "_", filename)
    return cleaned_filename

# Fonction pour télécharger une image à partir d'un lien et l'enregistrer dans le dossier
def download_image(row, output_folder):
    try:
        image_name = f"{row['identification']}_{row['Nom']}"  # Utiliser l'identification dans le nom du fichier
        image_url = row['link']
        response = requests.get(image_url)
        if response.status_code == 200:
            # Nettoyer le nom du fichier
            cleaned_image_name = clean_filename(image_name)
            filename = os.path.join(output_folder, f"{cleaned_image_name}.jpg")
            df.at[row.name, 'file_name'] = filename  # Utiliser l'index pour mettre à jour la valeur dans le DataFrame
            with open(filename, 'wb') as file:
                file.write(response.content)
            print(f"Image téléchargée : {filename}")
        else:
            print(f"Échec du téléchargement de l'image à partir de {image_url}. Statut {response.status_code}")
    except Exception as e:
        print(f"Erreur lors du téléchargement de l'image à partir de {image_url}: {e}")

# Assurez-vous qu'il n'y a pas de valeurs manquantes ou de doublons dans la colonne 'identification'
df = df.dropna(subset=['identification']).drop_duplicates(subset=['identification'])

# Triez le DataFrame en fonction de la colonne 'identification'
df.sort_values(by='identification', inplace=True)

# Réinitialisez l'index du DataFrame
df.reset_index(drop=True, inplace=True)

# Ajouter une colonne 'file_name' vide au DataFrame
df['file_name'] = ''

# Appliquez la fonction pour chaque ligne du DataFrame
for index, row in df.iterrows():
    download_image(row, output_folder)

# Vérifiez si la colonne 'file_name' a été ajoutée
print(df.head())


Image téléchargée : c:\Users\kiemd\OneDrive\Desktop\Porjetfin\images2\telephone_tablette\0_Music_Pioneer_F9_Oreillette_Bluetooth_St_r_o_BT5.3___Power_Banks_2000mAh_-_Noir.jpg
Image téléchargée : c:\Users\kiemd\OneDrive\Desktop\Porjetfin\images2\telephone_tablette\1_Music_Pioneer__couteur_Bluetooth_M10_V5.3_9D_St_r_o_-_2000mAh.jpg
Image téléchargée : c:\Users\kiemd\OneDrive\Desktop\Porjetfin\images2\telephone_tablette\2_P47_Casque_Audio_Bluetooth_-_P47___EDR_-_Version_2022_-_couleur_Noir.jpg
Image téléchargée : c:\Users\kiemd\OneDrive\Desktop\Porjetfin\images2\telephone_tablette\3_Kgtel_1280_T_l_phone_KGT_t_l_double_sim_original.jpg
Image téléchargée : c:\Users\kiemd\OneDrive\Desktop\Porjetfin\images2\telephone_tablette\4_Samsung_Galaxy_A03_Core_-_RAM_2Go_-_ROM_32Go_-_Onyx.jpg
Image téléchargée : c:\Users\kiemd\OneDrive\Desktop\Porjetfin\images2\telephone_tablette\5_Tecno_Spark_10c_-_6.6__-_ROM_128Go_-_RAM_4Go_-_Photo_50MP_-_5000mAh-_Dual_SIM___Garantie_12_Mois_-_Noir_-_Garantie_12_mois

In [9]:
df.head()

,Nom,prix,link,marque,Description,modele,couleur,cathegorie,identification,file_name
0,Music Pioneer F9 Oreillette Bluetooth Stéréo B...,2900,https://sn.jumia.is/unsafe/fit-in/300x300/filt...,Music Pioneer,"['Type de produit: Oreillette Bluetooth', 'Mar...",F9,Noir,telephone-tablette,0,c:\Users\kiemd\OneDrive\Desktop\Porjetfin\imag...
1,Music Pioneer Écouteur Bluetooth M10 V5.3 9D S...,2900,https://sn.jumia.is/unsafe/fit-in/300x300/filt...,Music Pioneer,"['Marque: Music Pioneer', 'Modèle: M10', 'Vers...",M10,Noir,telephone-tablette,1,c:\Users\kiemd\OneDrive\Desktop\Porjetfin\imag...
2,P47 Casque Audio Bluetooth - P47 + EDR - Versi...,2500,https://sn.jumia.is/unsafe/fit-in/300x300/filt...,P47,"['Version Bluetooth\xa0: ISSC 4.1 + EDR.', 'Pr...",Casque Audio Bluetooth - P47,Noir,telephone-tablette,2,c:\Users\kiemd\OneDrive\Desktop\Porjetfin\imag...
3,Kgtel 1280 Téléphone KGT tél double sim original,3990,https://sn.jumia.is/unsafe/fit-in/300x300/filt...,Kgtel,"['Produits téléphone KGtel', 'Marque -Kgtel', ...",MT 350,Noir,telephone-tablette,3,c:\Users\kiemd\OneDrive\Desktop\Porjetfin\imag...
4,Samsung Galaxy A03 Core - RAM 2Go - ROM 32Go -...,42900,https://sn.jumia.is/unsafe/fit-in/300x300/filt...,Samsung,"['Type Produit : Smartphone', 'Marque : Samsun...",A03 Core,onyx,telephone-tablette,4,c:\Users\kiemd\OneDrive\Desktop\Porjetfin\imag...


In [34]:
def normalize_and_resize_images(image_paths, target_size=(224, 224)):
    images = []
    for path in image_paths:
        img = Image.open(path)
        img = np.array(img) / 255.0
        img = Image.fromarray((img * 255).astype(np.uint8))
        img = img.resize(target_size, Image.BILINEAR)
        images.append(img)
    return images

In [30]:
import matplotlib.pyplot as plt

def plot_images(images, query, scores, n_row=2, n_col=5):
    _, axs = plt.subplots(n_row, n_col, figsize=(12, 12))
    axs = axs.flatten()
    for img, score, ax in zip(images, scores, axs):
        ax.set_title(f"Similarity: {score*100:.2f}%")
        ax.imshow(img)
    plt.show()


In [35]:
def calculate_similarity_with_names(text, image_data, image_paths, model_id="clip-ViT-B-32-multilingual-v1", top_k=4):
    # Utilisation du modèle SentenceTransformers
    model = SentenceTransformer(model_id)

    # Encodage du texte
    text_embedding = model.encode([text], convert_to_tensor=True)

    # Encodage des images dans la base de données
    image_embeddings = model.encode(
        [np.array(img) for img in image_data],
        batch_size=128,
        convert_to_tensor=True,
        show_progress_bar=True,
    )

    # Calcul de la similarité
    scores = util.pytorch_cos_sim(text_embedding, image_embeddings)[0].cpu().detach().numpy()

    # Récupération des indices des images les plus similaires
    top_indices = np.argsort(-scores)[:top_k]

    # Affichage des résultats
    matched_images = [(image_paths[idx], image_data[idx]) for idx in top_indices]
    plot_images([img_data for _, img_data in matched_images], text, scores)

    # Retourner les noms des images correspondantes
    matched_image_names = [image_paths[idx] for idx in top_indices]
    return top_indices, scores, matched_images, matched_image_names

# Exemple d'utilisation avec un texte et des images
text_query = "Samsung Chargeur ultra rapide 65W, USB Type-C, bloc d'alimentation, adaptateur"
#text = clean_image_name(text_query)
image_folder = './data1_image/'
image_paths = glob.glob(f'{image_folder}*.jpg')

# Normaliser et calculer la similarité
indices, scores, matched_images, matched_image_names = calculate_similarity_with_names(text_query, normalize_and_resize_images(image_paths), image_paths)

# Imprimer les noms des images correspondantes
print("Noms des images correspondantes:")
for name in matched_image_names:
    print(name)


Batches: 0it [00:00, ?it/s]


RuntimeError: stack expects a non-empty TensorList

# code pour extraire les links

In [23]:
import re

def extract_number_from_path(path, prefix_to_remove):
    # Utiliser une expression régulière pour supprimer le préfixe spécifié du chemin
    match = re.match(fr'{re.escape(prefix_to_remove)}(\d+)', path)
    
    if match:
        # Récupérer le nombre trouvé dans le groupe correspondant
        number = int(match.group(1))
        return number
    else:
        # Retourner une valeur par défaut ou signaler une absence de nombre
        return None

# Exemple d'utilisation
image_paths = matched_image_names  # Remplacez cela par votre liste de liens
prefix_to_remove = "./images2/telephone_tablette\\"

# Liste pour stocker les chiffres extraits
numbers = []

# Boucle sur tous les liens
for path in image_paths:
    number = extract_number_from_path(path, prefix_to_remove)
    numbers.append(number)

# Imprimer les chiffres extraits
print("Chiffres extraits :")
for number in numbers:
    print(number)


Chiffres extraits :
500
833
1183
19
